In [1]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os

from synthetic_data.tasks.writing import _get_gutenberg_subset, find_valid_chunks

gutenberg_pq: pl.DataFrame = _get_gutenberg_subset(2)

In [ ]:
gutenberg_pq.columns

In [101]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from IPython.display import Markdown

from synthetic_data.tasks.writing import _get_chunks_gutenberg

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

for row in gutenberg_pq.sample(100).iter_rows(named=True):
    chunks = _get_chunks_gutenberg(row, tiktoken_encoder)
    display(Markdown(f"**{row['title']}**"))
    for chunk in chunks:
        n_tokens = sum([len(tiktoken_encoder.encode(par)) for par in chunk])
        print("-"*100)
        print(f"n_tokens: {n_tokens}")
        for par in chunk:
            tokens = tiktoken_encoder.encode(par)
            display(Markdown(par))
        break


**Harper's Young People, March 22, 1881: An Illustrated Weekly**

**New Poems**

**What's Mine's Mine — Volume 3**

----------------------------------------------------------------------------------------------------
n_tokens: 629


I cannot say NO worldly pride mingled in the chief's contempt for
the distiller's money; his righteous soul was not yet clear of its
inherited judgments as to what is dignified and what is not. He had
in him still the prejudice of the landholder, for ages instinctive,
against both manufacture and trade. Various things had combined to
foster in him also the belief that trade at least was never free
from more or less of unfair dealing, and was therefore in itself a
low pursuit. He had not argued that nothing the Father of men has
decreed can in its nature be contemptible, but must be capable of
being nobly done. In the things that some one must do, the doer
ranks in God's sight, and ought to rank among his fellow-men,
according to how he does it. The higher the calling the more
contemptible the man who therein pursues his own ends. The humblest
calling, followed on the principles of the divine caller, is a true
and divine calling, be it scavenging, handicraft, shop-keeping, or
book-making. Oh for the day when God and not the king shall be
regarded as the fountain of honour.

...

**The Mirror of Literature, Amusement, and Instruction. Volume 14, No. 381, July 18, 1829**

----------------------------------------------------------------------------------------------------
n_tokens: 1292


Among so many crude and discordant opinions, I shall endeavour to
substitute another more consistent with the true etymology of the
word. I agree with the historian of Manchester, that the Roman
stations were prior to the roads, and that the latter were only the
channels of communication to the former. The stations commenced during
the conquest of the country, and all of them were completed at the
conclusion of it. The roads therefore could not be constructed till
the first or second summer after the stations were established.
Whoever has attentively observed the line or direction of the Watling
Street, must be convinced of the truth of the foregoing observations;
and the deviation from a straight line, which in many parts is so
apparent, and so evidently made to enable the Romans to pass from one
station to another, may be considered conclusive upon this point. I
therefore have no hesitation in asserting, that the Watling Street Way
is a Roman road, and probably planned and formed by Vespasian, the
celebrated Roman general in Britain, who named this road in compliment
to the emperor, _Vitellius, Vitellii Strata Via_, Watling Street Way.
Suetonius, in his _Life of Vespasian_, says, (chapter 4,) "_Claudio
principe, Narcissi gratiâ, legatus in Germaniam missus est
(Vespasianus;) inde in Britanniam translatus, tricies cum hoste
conflixit. Duas validissimas gentes, superq viginti oppida, et insulam
Vectam Britanniae proximam, in deditionem redegit, partim Auli Plautii
legati, partim Claudii ipsius ductu. Quare triumphalia ornamenta, et
in spatio brevi, duplex sacerdotium accepit, praeterea consulatum,
quem gessit per duos novissimos anni menses." Or, "In the reign of
Claudius, by the interest of Narcissus,[2] he (Vespasian) was sent
lieutenant general of a legion into Germany, from whence being removed
into Britain, he engaged the enemy in thirty distinct battles, and
subjected to the power of the Romans two very strong nations, and
above twenty great towns, and the Isle of Wight, upon the coast of
Britain, partly under the command of Aulus Plautius, and partly under
that of Claudius himself. In reward for these noble services he
received the triumphal ornaments, and in a short time after, two
priest's offices, besides the consulship, which he held for the two
last months of the year."

The same author, in his Life of Vitellius, seems to strengthen or
rather establish the conjecture of its being the _Vitellii Strata
Via_, for he says, (chapter 1,) "_indicia, stirpis (Vitelliorum) diu
mansisse, Viam Vitelliam ab Janiculo ad mare usque, item coloniam
ejusdem nominis._" Or, "Some monuments of the family continued a long
time, as the _Vitellian Way_, reaching from the Janiculum to the sea,
and likewise a colony of that name." From the abovementioned extracts,
it seems not improbable that one of the thirty battles mentioned by
Suetonius, might have been fought during the time the Romans were
forming this road through the Forest of Arden, which extended from
Henley, in Warwickshire, to Market Harborough, in Leicestershire; and
that it was called in compliment to Vitellius, the _Vitellian Way_,
afterwards corrupted to the _Watling Way_.

This road from the Avon, which it passes at Dove Bridge, to the Anker,
near Atherstone, forms the boundary between the counties of Leicester
and Warwick. In the month of June, 1824, numerous skulls and bones
were discovered in a line from the intersection of the road that leads
from Rugby to Lutterworth, with the Watling Street to Benones or
Bensford Bridge, the distance not being more than half a mile. These
bones were lying about two feet below the surface of the ground. Many
fragments of shields, spear heads, knives, and a sword,[3] placed by
the side of a skeleton, and at one end touching a funereal urn,[4] and
likewise several drinking cups, or small vessels, apparently formed of
half-baked clay, with clasps both of silver and brass, were found
within the abovementioned distance. On the contrary side of the road
were discovered beads, glass, and amber, but neither urns,
spear-heads, or fragments of shields; these relics, therefore,
probably belonged to the Britons, who fell encountering the Romans, to
prevent their forming a road through the Forest of Arden. There can be
little doubt of a battle having been here fought, from the bones,
urns, and tumuli discovered here and in the adjacent neighbourhood.
"In this parish (Church Over,") says Dugdale, "upon the old Roman Way,
called Watling Strete, is to be seen a very great tumulus, which is of
that magnitude, that it puts travellers beside the usual road," and a
_Letter_ from Elias Ashmole to Sir Wm. Dugdale,[5] states, "that about
a mile from hence (that is from Holywell Abbey, now the site of Caves
Inn,) there is a tumulus raised in the very middle of the high way,
which methought was worth observing." This tumulus, in an ancient
deed, is called the Pilgrim's Low. It was removed in making the
turnpike-road from Banbury to Lutterworth, about the year 1770. In the
plantations of Abraham Grimes, Esq., within half a mile of the site of
the former, is another tumulus of smaller dimensions, adjoining the
road which leads from Rugby to Lutterworth.

In [ ]:
import re

from synthetic_data.utils import ldictl
from synthetic_data.tasks.writing import GutenbergBacktranslation
first_sample = gutenberg_pq.sample(n=1)
task = GutenbergBacktranslation()

input = first_sample.to_dicts()
formatted_out = task.format_input_conversation(ldictl(input))

In [ ]:
formatted_out[0]